# Week 06

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Q1

In [2]:
spark = SparkSession.builder.appName("DecisionTreeModel").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/04 09:39:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv('covtype.csv', header=False, inferSchema=True)
df

DataFrame[_c0: int, _c1: int, _c2: int, _c3: int, _c4: int, _c5: int, _c6: int, _c7: int, _c8: int, _c9: int, _c10: int, _c11: int, _c12: int, _c13: int, _c14: int, _c15: int, _c16: int, _c17: int, _c18: int, _c19: int, _c20: int, _c21: int, _c22: int, _c23: int, _c24: int, _c25: int, _c26: int, _c27: int, _c28: int, _c29: int, _c30: int, _c31: int, _c32: int, _c33: int, _c34: int, _c35: int, _c36: int, _c37: int, _c38: int, _c39: int, _c40: int, _c41: int, _c42: int, _c43: int, _c44: int, _c45: int, _c46: int, _c47: int, _c48: int, _c49: int, _c50: int, _c51: int, _c52: int, _c53: int, _c54: int]

In [4]:
df.show()

24/09/04 09:39:45 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+---+---+---+---+----+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
| _c0|_c1|_c2|_c3|_c4| _c5|_c6|_c7|_c8| _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|_c27|_c28|_c29|_c30|_c31|_c32|_c33|_c34|_c35|_c36|_c37|_c38|_c39|_c40|_c41|_c42|_c43|_c44|_c45|_c46|_c47|_c48|_c49|_c50|_c51|_c52|_c53|_c54|
+----+---+---+---+---+----+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|2596| 51|  3|258|  0| 510|221|232|148|6279|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0| 

In [5]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: integer (nullable = true)
 |-- _c25: integer (nullable = true)
 |-- _c26: integer (nullable = true)
 |-- _

In [6]:
df.describe().show()

+-------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------

### Q2

In [7]:
numeric_cols = [col for col, dtype in df.dtypes if dtype == 'double']

In [8]:
mean_values = {col: df.select(col).na.drop().agg({col: 'mean'}).first()[0] for col in numeric_cols}

In [9]:
df_filled = df.fillna(mean_values)

In [10]:
categorical_columns = [col for col, dtype in df.dtypes if dtype == 'string']
indexers = [StringIndexer(inputCol=column, outputCol=column + "_index").fit(df_filled) for column in categorical_columns]

In [11]:
for indexer in indexers:
    df_filled = indexer.transform(df_filled)

In [12]:
df_filled.show()

+----+---+---+---+---+----+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
| _c0|_c1|_c2|_c3|_c4| _c5|_c6|_c7|_c8| _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|_c27|_c28|_c29|_c30|_c31|_c32|_c33|_c34|_c35|_c36|_c37|_c38|_c39|_c40|_c41|_c42|_c43|_c44|_c45|_c46|_c47|_c48|_c49|_c50|_c51|_c52|_c53|_c54|
+----+---+---+---+---+----+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|2596| 51|  3|258|  0| 510|221|232|148|6279|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0| 

### Q3

In [13]:
feature_columns = [col for col in df_filled.columns if col != 'label']

In [14]:
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

In [15]:
df_features = assembler.transform(df_filled)

In [16]:
train_df, test_df = df_features.randomSplit([0.8, 0.2], seed=1234)

In [17]:
dt = DecisionTreeClassifier(labelCol='_c54', featuresCol="features")

In [18]:
model = dt.fit(train_df)

24/09/04 09:40:00 WARN MemoryStore: Not enough space to cache rdd_55_14 in memory! (computed 1030.0 KiB so far)
24/09/04 09:40:00 WARN MemoryStore: Not enough space to cache rdd_55_7 in memory! (computed 1549.4 KiB so far)
24/09/04 09:40:00 WARN MemoryStore: Not enough space to cache rdd_55_5 in memory! (computed 1549.4 KiB so far)
24/09/04 09:40:00 WARN MemoryStore: Not enough space to cache rdd_55_11 in memory! (computed 1549.4 KiB so far)
24/09/04 09:40:00 WARN MemoryStore: Not enough space to cache rdd_55_2 in memory! (computed 1030.0 KiB so far)
24/09/04 09:40:00 WARN MemoryStore: Not enough space to cache rdd_55_6 in memory! (computed 1030.0 KiB so far)
24/09/04 09:40:00 WARN MemoryStore: Not enough space to cache rdd_55_8 in memory! (computed 1030.0 KiB so far)
24/09/04 09:40:00 WARN MemoryStore: Not enough space to cache rdd_55_17 in memory! (computed 1549.4 KiB so far)
24/09/04 09:40:00 WARN MemoryStore: Not enough space to cache rdd_55_1 in memory! (computed 1549.4 KiB so far

### Q4

In [19]:
predictions = model.transform(test_df)

In [20]:
evaluator = MulticlassClassificationEvaluator(labelCol="_c54", predictionCol="prediction", metricName="accuracy")

In [21]:
accuracy = evaluator.evaluate(predictions)

In [22]:
accuracy

0.9954964651987841

In [23]:
evaluator.setMetricName("weightedPrecision")
precision = evaluator.evaluate(predictions)

In [24]:
precision

0.9919654112815633

In [25]:
evaluator.setMetricName("weightedRecall")
recall = evaluator.evaluate(predictions)

In [26]:
recall

0.9954964651987842

In [27]:
spark.stop()